### Часть 1. Программирование на Python

Ссылка на контест:

http://45.77.142.102/cgi-bin/new-client?contest_id=12

### Часть 2. Проверка статистических гипотез

На платформе Delivery Club рестораны с быстрой скоростью доставки (готовка блюда + доставка блюда курьером до клиента), как правило - это рестораны, которые пользуются наибольшей популярность среди пользователей и получают наивысшую оценку (5 баллов). 

Дан датасет **quality_restaurants.csv**, который преставляет из себя выборку с характеристиками некоторых ресторанов платформы DC. 

*avg_cook_time* - среднее время готовки ресторана, 

*avg_rest_toclient_time* - среднее время доставки блюда курьером ресторана до клиента, 

*rating* - бальная оценка ресторана (средний пользовательский рейтинг).

*restaurant_id* - идентификатор ресторана

*city_id* - идентификатор города

### Задание 1

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('quality_restaurants.csv', index_col='restaurant_id', sep=',')
df

,avg_cook_time,avg_rest_toclient_time,rating,city_id
restaurant_id,,,,
0,1.456798e+01,1.882925e+01,5,1
1,2.663857e+00,2.358426e+00,3,3
2,1.106665e+01,1.300000e+01,1,3
3,1.484324e+01,1.267024e+01,5,3
4,8.097093e+00,2.112668e+01,3,2
...,...,...,...,...
297,2.187737e+01,5.600239e+00,5,3
298,1.055604e+01,6.177070e+00,1,2
299,1.000000e+01,1.626414e+01,4,2


Есть ли разница между **средним временем доставки ресторанов с рейтингом 5 звезд, 4 звезды, меньше 4 звезд.** При оценке использовать непараметрический критерий.

In [3]:
top1 = df[df.rating == 5]
top2 = df[df.rating == 4]
other = df[df.rating < 4]

$H_0$: Распределения трех выборок равны - нет разницы\
$H_1$: $H_0$ неверна - есть разница


Используем непараметрический критерий Краскела — Уоллиса (обобщения критерия Манна — Уитни) для множества несвязанных выборок

In [4]:
from scipy.stats import kruskal

In [9]:
stat, p = kruskal(top1.avg_rest_toclient_time, top2.avg_rest_toclient_time, other.avg_rest_toclient_time)

print(f'Stat={stat}, p-value={p}')

print("Разницы нет" if p > 0.05 else 'Разница есть')

Stat=45.79900806604573, p-value=1.1346757155270986e-10
Разница есть


### Задание 2


Для **города 3** рассчитать силу взаимосвязи между среднем временем готовки ресторана и среднем временем доставки. Позволяют ли полученные данные говорить о существовании связи? 

In [10]:
city = df[df.city_id == 3]
city

,avg_cook_time,avg_rest_toclient_time,rating,city_id
restaurant_id,,,,
1,2.663857e+00,2.358426e+00,3,3
2,1.106665e+01,1.300000e+01,1,3
3,1.484324e+01,1.267024e+01,5,3
11,1.361751e+01,2.148637e+01,5,3
17,5.334727e+00,1.300000e+01,2,3
...,...,...,...,...
292,1.921343e+01,1.766494e+01,5,3
294,3.597857e+00,2.872979e+00,1,3
295,1.301986e+01,4.721711e+00,5,3


Интерпретируем задачу иначе: \
    Существует ли корреляция между двумя случайными величинами?

$H_0$: данные независимы \
$H_1$: иначе

In [15]:
from scipy.stats import spearmanr, pearsonr, shapiro

Проверим данные на нормальность

In [18]:
shapiro(city.avg_cook_time), shapiro(city.avg_rest_toclient_time)

(ShapiroResult(statistic=0.06995248794555664, pvalue=3.158851499994602e-23),
 ShapiroResult(statistic=0.06994974613189697, pvalue=3.158611686279415e-23))

Данные не являются нормальными при $\alpha = 0.05$. Вместо Пирсона будем пользоваться Спирменом для выявления корреляции:

In [20]:
spearmanr(city.avg_cook_time, city.avg_rest_toclient_time)

SpearmanrResult(correlation=0.16558525059385773, pvalue=0.08384680668275912)

При $\alpha = 0.05$ мы должны принимать $H_0$, так как $p>\alpha$. Но обратим внимание, что
при чуть бОльшем $\alpha$, например $0.1$ можно уже утверждать, что зависимость есть. \
Попробуем другой тест, а именно Тау Кендалла:

In [21]:
from scipy.stats import kendalltau

In [22]:
kendalltau(city.avg_cook_time, city.avg_rest_toclient_time)

KendalltauResult(correlation=0.11171507359823139, pvalue=0.0843764304076722)

Опять же, pvalue не дает четких результатов. Чисто интуитивно хочется сказать, что все-таки скорость приготовления влияет на время доставки, но здесь все зависит от выбранного уровня значимости. \
При $\alpha = 0.1$ зависимость есть \
При $\alpha = 0.05$ данные независимы \
Думаю, что правильный ответ на поставленный вопрос это нет,
полученные данные не позволяют говорить о существовании взаимосвязи.

### Часть 3. SQL

Дана таблица **ORDERS** со следующими столбцами: id (идентификатор заказа), city_id (идентификатор города), user_id (идентификатор клиента).
Также дана таблица **CITIES**, поля - id (идентификатор города), city_name (название города).
Задача - написать SQL-запрос, отвечающий на следующий вопрос: сколько уникальных клиентов выполнили заказ в городах на букву П.

In [8]:
"""SELECT COUNT(user_id) FROM 
    (SELECT distinct user_id FROM
        orders JOIN CITIES ON ORDERS.city_id=CITIES.id
            WHERE city_name LIKE 'П%');"""